In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import MinMaxScaler

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV
from sklearn.model_selection import train_test_split

#모델링

from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

In [4]:
train=pd.read_csv('train.csv', index_col=0)
test=pd.read_csv('test_x.csv', index_col=0)
submission=pd.read_csv('sample_submission.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(45532, 77)
(11383, 76)
(11383, 1)


In [5]:
X_train = train.drop('voted', axis = 1)
y_train = train['voted']

In [6]:
drop_val = ['QaA', 'QbA', 'QcA','QdA', 'QeA',
       'QfA', 'QgA', 'QhA', 'QiA', 'QjA',
       'QkA', 'QlA', 'QmA', 'QnA', 'QoA',
       'QpA', 'QqA', 'QrA', 'QsA', 'QtA']

train = X_train.drop(drop_val, axis = 1)
test = test.drop(drop_val, axis = 1)

In [7]:
train_dummies = pd.get_dummies(train)
test_dummies = pd.get_dummies(test)
new_train=pd.DataFrame(train_dummies).to_numpy()
new_test=pd.DataFrame(test_dummies).to_numpy()

In [8]:
scaler=MinMaxScaler()
scaler.fit(new_train)
new_train=scaler.transform(new_train)
new_test=scaler.transform(new_test)

In [23]:
# cv =4

# lgbm_clf = LGBMClassifier()
ada_clf = AdaBoostClassifier()
params = {
         'n_estimators':[50,100,150],
         'learning_rate':[0.2,0.5,1.0,1.5]
         }
# base_estimators:None/

# cv는 4으로 지정
gridcv = GridSearchCV(ada_clf, param_grid=params, cv=4)
gridcv.fit(new_train, y_train)

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)

GridSearchCV 최적 파라미터: {'learning_rate': 0.5, 'n_estimators': 100}


In [21]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
params = {
    'max_leaf_nodes':list(range(2,100)),
                         'min_samples_split':[2,3,4]
}
grid_search_cv = GridSearchCV(tree_clf,
                             param_grid=params,verbose=1,cv=4)
grid_search_cv.fit(new_train,y_train)
print('Grid최적',grid_search_cv.best_params_)

Fitting 4 folds for each of 294 candidates, totalling 1176 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1176 out of 1176 | elapsed: 14.7min finished


Grid최적 {'max_leaf_nodes': 40, 'min_samples_split': 2}


In [22]:
tree_model = DecisionTreeClassifier(max_depth=100,max_leaf_nodes=40,min_samples_split=2)

In [ ]:
AdaBoostClassifier(base_estimator= tree_model, learning_rate=0.5, n_estimators=100)

In [ ]:
_clf = Classifier()

params = {}


# cv는 3으로 지정
gridcv = GridSearchCV(_clf, param_grid=params, cv=4)
gridcv.fit(new_train, y_train, early_stopping_rounds=100, eval_metric="auc",eval_set=(new_train, y_train))

print('GridSearchCV 최적 파라미터:', gridcv.best_params_)